In [1]:
from mnemonic import Mnemonic
from fastecdsa import keys, curve
import binascii
import hashlib
import hmac


In [2]:
"""here i create a class to store data in a tree like structure"""

class TreeNode():
    
    def __init__(self,parent):
        self.parent=parent
        self.pvt_key=None
        self.pub_key=None
        self.chain_code =None
        self.children = []
    

In [3]:
"""Here I define the function needed to generate the seed which is going to be used later """

def generate_seed(language,passphrase,strenght):
   mnemo = Mnemonic(language)
   words = mnemo.generate(strength=strenght)
   seed = mnemo.to_seed(words, passphrase=passphrase)
   print("Entropy=" + str( binascii.hexlify(mnemo.to_entropy(words)))) #prints the entropy in hex
   print("Words= " + words)   
   print("Seed = "+str(binascii.hexlify(seed))) #prints the seed in hex 
   return seed
   


In [4]:
seed=generate_seed("english","test",128)

Entropy=b'400dfb7fcac0ec5b4a5794e931d25690'
Words= divorce hurt that night attract color citizen verb truck model noble cage
Seed = b'57cfb012f7a62fc5ea951478e90581ffa083c4fe968cbb861fa0738d531dd9aebb20746ee7f67f5e9b116ae695a72064ca5ebb680894db29452100ff43042d3f'


In [5]:
def print_info(dictio):
    for keys,values in dictio.items():
        print(keys.upper())
        print(values)
   

In [6]:
"""In this section I derive from the seed the master private key and the master chain code, the private key is the one used to have control over your bank account represented by the public key. The bitcoin address is generated/corresponds from the public key"""
"""def get_master_info(seed):
    to_return={}
    master= hmac.new(seed,None,hashlib.sha512).hexdigest()
    master_pvt_key=master[:64]
    master_chain_code=master[64:]
    pvt=int(master_pvt_key, 16)
    master_pub_key = keys.get_public_key(pvt,curve.secp256k1)
    to_return["master_pvt_key"] = master_pvt_key
    to_return["master_pub_key"] = master_pub_key
    to_return["master_chain_code"] = master_chain_code
    return to_return"""

def get_master_info(seed):
    master=TreeNode(None)
    master_hmac= hmac.new(seed,None,hashlib.sha512).hexdigest()
    master_pvt_key=master_hmac[:64]
    master.pvt_key=master_pvt_key
    master_chain_code=master_hmac[64:]
    master.chain_code=master_chain_code
    pvt=int(master_pvt_key, 16)
    master_pub_key = keys.get_public_key(pvt,curve.secp256k1)
    pub_key= hex(master_pub_key.y)[2:4]+hex(master_pub_key.x)[2:] #followed the guide https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
    master.pub_key=pub_key
    return master


In [7]:
master_info=get_master_info(seed)
master_info.pub_key


'fcb4d1491a2d19bbec428f85b9d1dbb94ae305cdd5ca200cb9af217cebcee408cd'

In [8]:
"""In this section I define a function to derive child addresses using hardened derivation"""

def create_hardened_children(parent,index,num): #the index should be passed as a 32 bit hex
  
    for i in range(0,num): 
        child=TreeNode(parent.pvt_key)
        input_bytearray=str.encode(parent.pvt_key)+str.encode(parent.chain_code)+str.encode(hex(int(index, 16)+i)[2:])
        child_info=hmac.new(input_bytearray,None,hashlib.sha512).hexdigest()
        child.pvt_key=child_info[:64]
        child.chain_code=child_info[64:]
        pvt=int(child.pvt_key, 16)
        child_pub_key=keys.get_public_key(pvt,curve.secp256k1)
        pub_key= hex(child_pub_key.y)[2:4]+hex(child_pub_key.x)[2:] #followed the guide https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
        child.pub_key=pub_key
        parent.children.append(child)
            

In [9]:
"""in this section I create a children from the public key of the master"""

def create_children(parent,index,num): #the index should be passed as a 32 bit hex
  
    for i in range(0,num): 
        child=TreeNode(parent.pvt_key)
        input_bytearray=str.encode(parent.pub_key)+str.encode(parent.chain_code)+str.encode(hex(int(index, 16)+i)[2:])
        child_info=hmac.new(input_bytearray,None,hashlib.sha512).hexdigest()
        child.pvt_key=child_info[:64]
        child.chain_code=child_info[64:]
        pvt=int(child.pvt_key, 16)
        child_pub_key=keys.get_public_key(pvt,curve.secp256k1)
        pub_key= hex(child_pub_key.y)[2:4]+hex(child_pub_key.x)[2:] #followed the guide https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
        child.pub_key=pub_key
        parent.children.append(child)

In [10]:
create_children(master_info,"00000000",2)
create_hardened_children(master_info,"80000000",1)

for i in master_info.children:
        print("Private Key " + i.pvt_key)
        print("Public Key " + i.pub_key)
        print("Chain Code " + i.chain_code)
        print("-------------------------------------------------------------------------")
        

Private Key 667656b9c7b9a02efac454c1a13a90029616eb868674a960af2a29a50e893031
Public Key ff36f2fc726dd3d3afcbdd7b0dc1081c10753adaacd2cee9bd94f3b8b484c34017
Chain Code 4acb6f31d1977f55524186cf17af8660a3fd939d4934905126be4f636db94c9d
-------------------------------------------------------------------------
Private Key 5c483fca4e4b9f567f3dcb59228c71b3e756c471f662264c8960045e5a0fac21
Public Key 118196ece136e37678573aa3b9d79d03b03eb79362ce31ca153065c27cf1f85cdf
Chain Code 5c21069a3706cbab61768e677b3e62c8f879b1b7d10a42f75561d13a7227f3e6
-------------------------------------------------------------------------
Private Key d415e213bc9d2baff674edf283af38f2930a138411ca10e79d4a61590d5f2cf6
Public Key 22f32071aeef394eb7aa89d9b3f3144417b2b7f5ebe3457ccace209f5920cceaa
Chain Code a7ec17ac27f7ce91d3e9c1375cca4c99aef01c025a34e41f1e9ce92d7a386731
-------------------------------------------------------------------------


In [11]:
"""Lets try to create some grandkids"""

for i in master_info.children:
    create_hardened_children(i,"80000000",1)
    create_children(i,"00000000",2)
    for j in i.children:
        print("Private Key " + j.pvt_key)
        print("Public Key " + j.pub_key)
        print("Chain Code " + j.chain_code)
        print("-------------------------------------------------------------------------")
        




Private Key 9af914a15b146508ac9d822dc22ef30754f2e333c2b380951fa1339b0f7f762c
Public Key 121065d3fdacb959a6110283fbdf023357058b6ca207fd5b10e8c4fad2b97e762e
Chain Code 9a453f0e05c57e646a580c027797da16ab5d3796bc188fa81b72add6e19f1e82
-------------------------------------------------------------------------
Private Key 829be505ae10627d5122132a6645c15b74655b853761e7f339bb0d74d7d0f589
Public Key 8dd4ce5eec96abad635010406e6af72ee844f6a6288ba9ece69caa88101c60f06d
Chain Code ad3486848709d22aafb0a1878e7b1d7b6ffae9035d420c73e10fc03198227821
-------------------------------------------------------------------------
Private Key 7a543e1e95848f7301f1f20e82658c169dcb1006d98d2b826ccc527d30c0002b
Public Key 7a35316e79e198db1dd6feaad1ef8c4fe55354f5628ecf888c41d3da9c740b0517
Chain Code b435f2371a8561976e1fc50b4be4207c86b8f3b8427509b37fd3cb70e821b19d
-------------------------------------------------------------------------
Private Key a26b2849f1ded30e42c53c3d1e0d33662c11d179f11942e16bc8cc720dfa5c10
Public K

In [12]:
def retrieve_hardened_children(parent):
    for i in parent.children:
        if 

SyntaxError: invalid syntax (<ipython-input-12-44483769ee52>, line 3)